# Automatic differentiation

**References:**
- http://www.columbia.edu/~ahd2125/post/2015/12/5/
- https://stackoverflow.com/questions/36370129/does-tensorflow-use-automatic-or-symbolic-gradients

In [1]:
import tensorflow as tf

In [2]:
def my_func(w,x):
    
    f_0 = tf.exp(w[0,0] + w[0,1]*x)
    f_1 = tf.exp(w[1,0] + w[1,1]*f_0)
    f_2 = tf.exp(w[2,0] + w[2,1]*f_1)
    
    return f_0, f_1, f_2

In [3]:
def my_func(w,x):
    
    with tf.name_scope("f_0_level") as scope_0:
        f_0 = tf.exp(w[0,0], + w[0,1]*x)
        
    with tf.name_scope("f_1_level") as scope_1:
        f_1 = tf.exp(w[1,0], + w[1,1]*x)
        
    with tf.name_scope("f_2_level") as scope_2:
        f_2 = tf.exp(w[2,0], + w[2,1]*x)
        
    return f_0, f_1, f_2

In [ ]:
w_0 = np.vstack((np.zeroes(3)), np.ones(3) )